In [37]:
### Dont need this code anymore, but I want it!
import pandas as pd
from functools import reduce

# ---- 1. Read the original Excel file ----
file_in = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/GDP_forecast.xlsx"
df = pd.read_excel(file_in)

# Clean up column names (remove extra spaces etc.)
df.columns = df.columns.str.strip()

print("Original columns:")
print(df.columns.tolist())

# ---- 2. Define which date/value column belongs to which region (except US) ----
region_map = {
    "AU": {"date_col": "Date",    "value_col": "AU"},
    "GB": {"date_col": "Date.1",  "value_col": "GB BN_SURVEY_AVERAGE"},
    "CH": {"date_col": "Date.2",  "value_col": "CH YoY Growth"},
    "JP": {"date_col": "Date.3",  "value_col": "JP YoY Growth"},
    "EU": {"date_col": "Date.3",  "value_col": "EU YoY Growth"},  # same dates as JP
}

# Sanity check that the columns exist
for r, info in region_map.items():
    for c in (info["date_col"], info["value_col"]):
        if c not in df.columns:
            raise ValueError(f"Column '{c}' for region {r} not found in DataFrame!")

# ---- 3. Helper to build tidy DataFrame for one region (non-US) ----
def make_region_df(df, region, date_col, value_col):
    sub = df[[date_col, value_col]].copy()
    sub.columns = ["Date", region]

    # Parse dates
    sub["Date"] = pd.to_datetime(sub["Date"], errors="coerce")

    # Convert values to numeric (no extra percent scaling!)
    sub[region] = pd.to_numeric(sub[region], errors="coerce")

    # Drop rows without dates
    sub = sub.dropna(subset=["Date"])
    return sub

# ---- 4. Build region DataFrames for AU, GB, CH, JP, EU ----
region_dfs = []
for region, info in region_map.items():
    r_df = make_region_df(
        df,
        region=region,
        date_col=info["date_col"],
        value_col=info["value_col"],
    )
    region_dfs.append(r_df)

# ---- 5. Build US DataFrame from YEAR + QUARTER ----
us = df[["YEAR", "QUARTER", "US RGDP Annual Forecast"]].copy()

# Drop rows without year/quarter/value
us = us.dropna(subset=["YEAR", "QUARTER", "US RGDP Annual Forecast"])

# Ensure integers for YEAR/QUARTER
us["YEAR"] = us["YEAR"].astype(int)
us["QUARTER"] = us["QUARTER"].astype(int)

# Map quarter to first month of quarter: Q1→1, Q2→4, Q3→7, Q4→10
us["month"] = (us["QUARTER"] - 1) * 3 + 1

# Build a Date column at the start of the quarter
us["Date"] = pd.to_datetime(
    dict(year=us["YEAR"], month=us["month"], day=1),
    errors="coerce"
)

# Keep only Date and value, rename value column to 'US'
us = us[["Date", "US RGDP Annual Forecast"]].rename(
    columns={"US RGDP Annual Forecast": "US"}
)

# Convert US values to numeric
us["US"] = pd.to_numeric(us["US"], errors="coerce")
us = us.dropna(subset=["Date"])

region_dfs.append(us)

# ---- 6. Merge all regions on Date (outer join) ----
merged = reduce(
    lambda left, right: pd.merge(left, right, on="Date", how="outer"),
    region_dfs
)

# ---- 7. Sort by date and place on a monthly grid, forward-fill ----
merged = merged.sort_values("Date")

# Monthly frequency: month-start
monthly = (
    merged.set_index("Date")
          .resample("MS")     # Month Start
          .asfreq()
          .sort_index()
)

# Forward-fill missing months for all regions
monthly = monthly.ffill()

# Back to a regular column index
monthly = monthly.reset_index()

# Divide US, AU, GB by 100
cols_to_divide = ["US", "AU", "GB"]

for col in cols_to_divide:
    if col in monthly.columns:
        monthly[col] = monthly[col] / 100.0

GDP_forecast =monthly


# ---- 7. Sort by date and place on a monthly grid, forward-fill ----
merged = merged.sort_values("Date")

# Monthly frequency: month-start
monthly = (
    merged.set_index("Date")
          .resample("MS")     # Month Start
          .asfreq()
          .sort_index()
)

# Forward-fill missing months for all regions
monthly = monthly.ffill()

# Back to a regular column index
monthly = monthly.reset_index()

# Divide US, AU, GB by 100
cols_to_divide = ["US", "AU", "GB"]
for col in cols_to_divide:
    if col in monthly.columns:
        monthly[col] = monthly[col] / 100.0

# (Optional) Drop rows where ALL non-US regions are still NaN
non_us_regions = ["AU", "GB", "CH", "JP", "EU"]
monthly = monthly.dropna(subset=non_us_regions, how="all")

# 🔽 Sort so that the newest date is first
monthly = monthly.sort_values("Date", ascending=False).reset_index(drop=True)

GDPforecast = monthly

print(GDPforecast.head(10))

out_path = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/GDP_forecasts.csv"
GDPforecast.to_csv(out_path, index=False)



Original columns:
['Date', 'AU', 'YEAR', 'QUARTER', 'US RGDP Annual Forecast', 'date', 'GB BN_SURVEY_AVERAGE', 'Date.1', 'CH YoY Growth', 'Date.2', 'JP YoY Growth', 'Date.3', 'EU YoY Growth']
        Date        AU     GB        CH        JP        EU        US
0 2025-10-01  0.038000  0.025  0.012846  0.010747  0.013505  0.015632
1 2025-09-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
2 2025-08-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
3 2025-07-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
4 2025-06-01  0.021000  0.025  0.024395  0.019764  0.015220  0.013453
5 2025-05-01  0.021000  0.025  0.024395  0.019764  0.015220  0.013453
6 2025-04-01  0.021000  0.025  0.024395  0.019764  0.015220  0.013453
7 2025-03-01  0.024018  0.016  0.016688  0.018356  0.016217  0.021749
8 2025-02-01  0.024018  0.016  0.016688  0.018356  0.016217  0.021749
9 2025-01-01  0.024018  0.016  0.016688  0.018356  0.016217  0.021749


In [39]:
import pandas as pd
from functools import reduce

# ---- 1. Read the Inflation forecast file ----
file_in = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/INF_forecast.xlsx"
df = pd.read_excel(file_in)

# Clean up column names
df.columns = df.columns.str.strip()

print("Original columns:")
print(df.columns.tolist())

# ---- 2. Region mapping for non-US (they all use observation_date) ----
region_map = {
    "UK": {"date_col": "observation_date", "value_col": "UK"},
    "CH": {"date_col": "observation_date", "value_col": "CH"},
    "JP": {"date_col": "observation_date", "value_col": "JP"},
    "AU": {"date_col": "observation_date", "value_col": "AU"},
    "EU": {"date_col": "observation_date", "value_col": "EU"},
}

# Sanity check that the columns exist
for r, info in region_map.items():
    for c in (info["date_col"], info["value_col"]):
        if c not in df.columns:
            raise ValueError(f"Column '{c}' for region {r} not found in DataFrame!")

# ---- 3. Helper: build tidy DataFrame for one region (non-US) ----
def make_region_df(df, region, date_col, value_col):
    sub = df[[date_col, value_col]].copy()
    sub.columns = ["Date", region]

    # Parse dates
    sub["Date"] = pd.to_datetime(sub["Date"], errors="coerce")

    # Values are already in percentages; keep as-is (no division by 100 here)
    sub[region] = pd.to_numeric(sub[region], errors="coerce")

    # Drop rows without dates
    sub = sub.dropna(subset=["Date"])
    return sub

# ---- 4. Build DataFrames for UK, CH, JP, AU, EU ----
region_dfs = []
for region, info in region_map.items():
    r_df = make_region_df(
        df,
        region=region,
        date_col=info["date_col"],
        value_col=info["value_col"],
    )
    region_dfs.append(r_df)

# ---- 5. Build US DataFrame from YEAR + QUARTER ----
us = df[["YEAR", "QUARTER", "US Forecast"]].copy()

# Remove rows missing any of these
us = us.dropna(subset=["YEAR", "QUARTER", "US Forecast"])

# Ensure integers for YEAR/QUARTER
us["YEAR"] = us["YEAR"].astype(int)
us["QUARTER"] = us["QUARTER"].astype(int)

# Map quarter to first month of quarter: Q1→1, Q2→4, Q3→7, Q4→10
us["month"] = (us["QUARTER"] - 1) * 3 + 1

# Construct Date as first day of quarter
us["Date"] = pd.to_datetime(
    dict(year=us["YEAR"], month=us["month"], day=1),
    errors="coerce"
)

# Keep only Date and value, rename value column to 'US'
us = us[["Date", "US Forecast"]].rename(columns={"US Forecast": "US"})

# Values already in percentages; keep as-is
us["US"] = pd.to_numeric(us["US"], errors="coerce")

# Drop rows without dates
us = us.dropna(subset=["Date"])

region_dfs.append(us)

# ---- 6. Merge all regions on Date (outer join) ----
merged = reduce(
    lambda left, right: pd.merge(left, right, on="Date", how="outer"),
    region_dfs
)

# ---- 7. Sort by date, resample monthly, forward-fill ----
merged = merged.sort_values("Date")

# Put on a monthly grid (Month Start)
monthly = (
    merged.set_index("Date")
          .resample("MS")   # Month Start
          .asfreq()
          .sort_index()
)

# Forward-fill missing months for all regions
monthly = monthly.ffill()

# Back to normal columns
monthly = monthly.reset_index()

# ---- 8. Save the rearranged table ----
file_out = "INF_forecast_rearranged.xlsx"
#monthly.to_excel(file_out, index=False)

# Divide US, AU, GB by 100
cols_to_divide = ['UK', 'CH', 'JP', 'AU', 'EU', 'US']

for col in cols_to_divide:
    if col in monthly.columns:
        monthly[col] = monthly[col] / 100.0

INFforecast = monthly
INFforecast.rename(columns={'UK': 'GB'}, inplace=True)

INFforecast = INFforecast.sort_values("Date", ascending=False).reset_index(drop=True)

print("Done. Rearranged table saved to:", file_out)
print(INFforecast)

out_path = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Inflation_forecasts.csv"
INFforecast.to_csv(out_path, index=False)


Original columns:
['YEAR', 'QUARTER', 'US Forecast', 'observation_date', 'UK', 'CH', 'JP', 'AU', 'EU']
Done. Rearranged table saved to: INF_forecast_rearranged.xlsx
          Date        GB        CH        JP     AU        EU      US
0   2025-10-01  0.038000  0.000100  0.030000  0.038  0.020789  0.0288
1   2025-09-01  0.041000  0.002000  0.029000  0.032  0.022124  0.0279
2   2025-08-01  0.041000  0.002000  0.027000  0.032  0.020289  0.0279
3   2025-07-01  0.042000  0.002000  0.031000  0.032  0.020079  0.0279
4   2025-06-01  0.041000  0.001000  0.033000  0.021  0.019676  0.0317
..         ...       ...       ...       ...    ...       ...     ...
665 1970-05-01  0.060836  0.031368  0.072456    NaN       NaN     NaN
666 1970-04-01  0.056188  0.025928  0.076836    NaN       NaN     NaN
667 1970-03-01  0.051420  0.024992  0.077171    NaN       NaN     NaN
668 1970-02-01  0.049307  0.021236  0.077513    NaN       NaN     NaN
669 1970-01-01  0.049574  0.023134  0.077513    NaN       NaN    

In [40]:
print(GDPforecast)
print(INFforecast)

          Date        AU     GB        CH        JP        EU        US
0   2025-10-01  0.038000  0.025  0.012846  0.010747  0.013505  0.015632
1   2025-09-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
2   2025-08-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
3   2025-07-01  0.032000  0.025  0.012846  0.010747  0.013505  0.014502
4   2025-06-01  0.021000  0.025  0.024395  0.019764  0.015220  0.013453
..         ...       ...    ...       ...       ...       ...       ...
659 1970-11-01  0.052632    NaN       NaN       NaN       NaN  0.029943
660 1970-10-01  0.052632    NaN       NaN       NaN       NaN  0.029943
661 1970-09-01  0.031579    NaN       NaN       NaN       NaN  0.029362
662 1970-08-01  0.031579    NaN       NaN       NaN       NaN  0.029362
663 1970-07-01  0.031579    NaN       NaN       NaN       NaN  0.029362

[664 rows x 7 columns]
          Date        GB        CH        JP     AU        EU      US
0   2025-10-01  0.038000  0.000100  0.0300

In [41]:
# 1) Work on copies
gdp = GDPforecast.copy()
inf = INFforecast.copy()

# 2) Ensure Date is datetime and sorted ascending
gdp["Date"] = pd.to_datetime(gdp["Date"])
inf["Date"] = pd.to_datetime(inf["Date"])

gdp = gdp.sort_values("Date")
inf = inf.sort_values("Date")

# 3) Align on common dates (inner join on Date)
gdp = gdp.set_index("Date")
inf = inf.set_index("Date")

common_dates = gdp.index.intersection(inf.index)

gdp = gdp.loc[common_dates].sort_index()
inf = inf.loc[common_dates].sort_index()

# 4) Regions we care about
regions = ["AU", "GB", "CH", "JP", "EU", "US"]

# 5) 12-month changes (Δ) in forecasts
gdp_change = gdp[regions].diff(12)   # ΔGDP_t = GDP_t − GDP_{t−12}
inf_change = inf[regions].diff(12)   # ΔINF_t = INF_t − INF_{t−12}

# 6) Brooks Business Cycle signal:
#    BC = 0.5 * ΔGDP − 0.5 * ΔINF
bc_values = 0.5 * gdp_change + 0.5 * inf_change

# 7) Build final table with Date as a column
bc_signal = bc_values.reset_index()   # Date becomes a column again

# Optionally save
bc_signal.to_csv(
    r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/BusinessCycle_Signal.csv",
    index=False
)

bc_signal = bc_signal.sort_values("Date", ascending=False).reset_index(drop=True)
print(bc_signal.head(15))


         Date        AU        GB        CH        JP        EU        US
0  2025-10-01  0.013753  0.012755 -0.006067  0.003253  0.000560  0.000613
1  2025-09-01  0.003915  0.015933 -0.006439  0.005382  0.004313  0.000263
2  2025-08-01  0.003915  0.013544 -0.007526  0.001882  0.001311  0.000263
3  2025-07-01  0.003915  0.014384 -0.008677  0.004882 -0.000921  0.000263
4  2025-06-01 -0.017144  0.010090  0.005644  0.017286  0.002249 -0.000699
5  2025-05-01 -0.017144  0.009557  0.004335  0.018286  0.001600 -0.000699
6  2025-04-01 -0.017144  0.008801  0.005117  0.020286  0.003941 -0.000699
7  2025-03-01 -0.012181 -0.008330  0.002249  0.019523  0.004341  0.004022
8  2025-02-01 -0.012181 -0.007199  0.001544  0.019523  0.004336  0.004022
9  2025-01-01 -0.012181 -0.007718  0.001376  0.024023  0.004364  0.004022
10 2024-12-01 -0.016273 -0.015126  0.002012  0.007721  0.003233  0.001534
11 2024-11-01 -0.016273 -0.015141  0.003891  0.003221  0.004899  0.001534
12 2024-10-01 -0.016273 -0.019988  0.0